In [2]:
import torch                    # pythorch 기본 라이브러리 ( 텐서 연산, 자동 미분 등)
import torch.nn as nn           # 신경망 레이어, 활성화/손실함수
import torch.optim as optim     # 가중치 업데이트 (최적화 알고리즘)

In [5]:
# 데이터 정의
X = [[0,0],[0,1],[1,0],[1,1]]   # 입력층 shape 4, 2
y = [[0],[1],[1],[0]]           # 출력층 shape 4, 1

In [6]:
X = torch.FloatTensor(X)  # 입력 데이터 Tensor
y = torch.FloatTensor(y)  # 정답 레이블 데이터 Tensor

print(X.shape)
print(y.shape)

torch.Size([4, 2])
torch.Size([4, 1])


In [ ]:
model = nn.Sequential(      # 이거 왜씀? 교육용으로 좋다 실험시 좋다, 못쓰는건 : 분기 구조, skip connection (ResNet), 여러 입력 / 여러 출력, 중간 출력이 필요한 경우
    nn.Linear(2, 4),     # Linear 쓰는이유 : 표 데이터, 간단한 문제 다른거 -  Conv2d / RNN, LSTM, GRU / MultiheadAttention,Transformer ...
    nn.Tanh(),           # 왜써 ? : 출력을 부드럽게 −1 ~ 1로 눌러서 비선형성을 주고, 학습을 안정시키기 위해
    nn.Linear(4, 2),
    nn.Tanh(),
    nn.Linear(2,1),
    nn.Sigmoid()        # Sigmoid 쓰는 이유 : 이진 분류 BCE loss와 궁합이 딱 맞음 , softmax
)

| 활성화 함수 | 특징 | 언제 씀 |
|-------------|------|---------|
| ReLU | 음수 0, 양수 그대로 | 은닉층 기본값 |
| LeakyReLU | 음수도 조금 통과 | ReLU 죽음 방지 |
| ELU | 음수 부드럽게 | 수렴 안정 |
| GELU | 확률적 ReLU | Transformer |
| SELU | self-normalizing | 특수 구조 |
| Sigmoid | 0~1 | 은닉층 ❌ |


| 문제 유형 | 출력층 마지막 | 손실함수 | 출력 의미 |
|----------|---------------|----------|-----------|
| 이진 분류 | **Sigmoid** | **BCE / BCEWithLogitsLoss** | P(class=1) |
| 다중 분류 (1개 선택) | **Softmax** | **CrossEntropyLoss** | 클래스 확률 |
| 멀티라벨 분류 | **Sigmoid (여러 개)** | **BCEWithLogitsLoss** | 클래스별 독립 확률 |
| 회귀 | **없음** | **MSE / MAE** | 연속값 |


In [10]:
# 손실함수, 옵티마이저
criterion = nn.BCELoss()

# 손실함수 : 얼마나 틀렸는지 알려줌
# 옵티마이저 : 손실이 줄어들도록 가중치를 어떻게, 얼마나 업데이트할지 결정하는 놈
optimizer = optim.SGD(  # 확률적 경사하강법 SGD
    model.parameters(), # 학습할 가중치와 편향
    lr = 0.1            # 학습률
)


| 옵티마이저 | 특징 | 언제 씀 |
|-----------|------|---------|
| SGD | 단순, 느림 | 이론 / 실험 |
| SGD+momentum | 안정 | 기본 SGD 개선 |
| Adam | 빠르고 편함 | 기본 선택 |
| AdamW | 정규화 우수 | 최신 모델 |
| RMSprop | 변동 완화 | RNN |
| Adagrad | 희소 데이터 | 특수 |


In [11]:
# 학습 루프
for epoch in range(10000):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0 :
        print(f"Epoch {epoch}, Loss : {loss.item() : .4f}")

# 결과 확인
with torch.no_grad():                   # 평가 단계에서는 gradient 계산 하지않음 (메모리 절약, 속도 향상)
    prediction = model(X).round()       # 모델 출력이 0~1 확률 값. round()는 반올림
    print("최종 예측 :", prediction)

Epoch 0, Loss :  0.7252
Epoch 1000, Loss :  0.0097
Epoch 2000, Loss :  0.0032
Epoch 3000, Loss :  0.0019
Epoch 4000, Loss :  0.0014
Epoch 5000, Loss :  0.0011
Epoch 6000, Loss :  0.0009
Epoch 7000, Loss :  0.0007
Epoch 8000, Loss :  0.0006
Epoch 9000, Loss :  0.0006
최종 예측 : tensor([[0.],
        [1.],
        [1.],
        [0.]])
